In [107]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [108]:
train_df = pd.read_csv("./train_fix.csv")
train_df_pre = pd.read_csv("./train_pre.csv")
test_df = pd.read_csv('test_fix.csv')
test_df_pre = pd.read_csv("./train_pre.csv")


print('The shape of train features is:', train_df.shape)
print('The shape of test features is:', test_df.shape)

The shape of train features is: (91531, 27)
The shape of test features is: (27859, 27)


In [109]:
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)
labels = np.array(train_df_pre['is_canceled'])
adrlabels = np.array(train_df_pre['adr'])
train_df = train_df.drop('market_segment_Undefined', axis = 1)
train_df = train_df.drop('distribution_channel_Undefined', axis = 1)
train_df = train_df.join(train_df_pre["previous_cancellations"])
train_df = train_df.join(train_df_pre["previous_bookings_not_canceled"])
test_df = test_df.join(test_df_pre["previous_cancellations"])
test_df = test_df.join(test_df_pre["previous_bookings_not_canceled"])
print(train_df.shape)
print(test_df.shape)

(91531, 43)
(27859, 43)


In [110]:
feature_list = list(train_df.columns)
print(feature_list)
feature_list = list(test_df.columns)
print(feature_list)

['stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'booking_changes', 'required_car_parking_spaces', 'total_of_special_requests', 'total_guests', 'total_nights', 'arrival_date_month_number', 'hotel_type', 'country_type', 'reserved_room_type_categorical', 'is_family', 'long_stay', 'is_weekend', 'meal_numerical', 'is_agent', 'is_company', 'lead_time', 'agent', 'company', 'market_segment_Aviation', 'market_segment_Complementary', 'market_segment_Corporate', 'market_segment_Direct', 'market_segment_Groups', 'market_segment_Offline TA/TO', 'market_segment_Online TA', 'distribution_channel_Corporate', 'distribution_channel_Direct', 'distribution_channel_GDS', 'distribution_channel_TA/TO', 'deposit_type_No Deposit', 'deposit_type_Non Refund', 'deposit_type_Refundable', 'customer_type_Contract', 'customer_type_Group', 'customer_type_Transient', 'customer_type_Transient-Party', 'previous_cancellations', 'previous_bookings_not_canceled']
['stays_in_weekend_nights

In [111]:
train_df = np.array(train_df)
test_df = np.array(test_df)
standardScalerX = StandardScaler()
train_df = standardScalerX.fit_transform(train_df)
test_df = standardScalerX.transform(test_df)
'''
total = len(train_df)
valid_len = int(total/4)
train_len = total-valid_len
trainX = train_df[:train_len]
validX = train_df[train_len:]
trainY = adrlabels[:train_len]
validY = adrlabels[train_len:]
'''
trainX, testX, trainY, testY = train_test_split(
    train_df, adrlabels, test_size=0.25, random_state=42)

In [112]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [113]:
'''
param_tuning = {
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.5, 0.7],
    'colsample_bytree': [0.5, 0.7],
    'n_estimators' : [100, 200, 500],
    'objective': ['reg:squarederror']
}

xgb_adr = XGBRegressor()
gsearch = GridSearchCV(estimator = xgb_adr,
                       param_grid = param_tuning,                        
                       #scoring = 'neg_mean_absolute_error', #MAE
                       #scoring = 'neg_mean_squared_error',  #MSE
                       cv = 5,
                       n_jobs = -1,
                       verbose = 1)

gsearch.fit(trainX, trainY)
print(gsearch.best_params_)
'''

"\nparam_tuning = {\n    'learning_rate': [0.01, 0.1],\n    'max_depth': [3, 5, 7, 10],\n    'subsample': [0.5, 0.7],\n    'colsample_bytree': [0.5, 0.7],\n    'n_estimators' : [100, 200, 500],\n    'objective': ['reg:squarederror']\n}\n\nxgb_adr = XGBRegressor()\ngsearch = GridSearchCV(estimator = xgb_adr,\n                       param_grid = param_tuning,                        \n                       #scoring = 'neg_mean_absolute_error', #MAE\n                       #scoring = 'neg_mean_squared_error',  #MSE\n                       cv = 5,\n                       n_jobs = -1,\n                       verbose = 1)\n\ngsearch.fit(trainX, trainY)\nprint(gsearch.best_params_)\n"

In [114]:
xgb_adr = XGBRegressor(
    objective = 'reg:squarederror',
    colsample_bytree = 0.7,
    learning_rate = 0.1,
    max_depth = 3,
    n_estimators = 200,
    subsample = 0.5)
xgb_adr.fit(trainX, trainY, verbose=False)
predict_adr = xgb_adr.predict(validX)
xgbRMSE = rmse(predict_adr, validY)
print("rmse is: " + str(xgbRMSE))

rmse is: 21.85676119203651


In [115]:
xgb_adr = XGBRegressor(
    objective = 'reg:squarederror',
    colsample_bytree = 0.7,
    learning_rate = 0.1,
    max_depth = 3,
    n_estimators = 200,
    subsample = 0.5)
xgb_adr.fit(train_df, adrlabels, verbose=False)
testpredict_xgb = xgb_adr.predict(test_df)
adrpredict_xgb = pd.DataFrame(testpredict_xgb,columns=['predict_adr'])
adrpredict_xgb.to_csv('adr_predict_mod.csv')